In [1]:
pip install langchain 


   ---------------------------------------- 0.0/798.0 kB ? eta -:--:--
    --------------------------------------- 10.2/798.0 kB ? eta -:--:--
   - ------------------------------------- 30.7/798.0 kB 660.6 kB/s eta 0:00:02
   -- ------------------------------------ 41.0/798.0 kB 393.8 kB/s eta 0:00:02
   -- ------------------------------------ 41.0/798.0 kB 393.8 kB/s eta 0:00:02
   -- ------------------------------------ 41.0/798.0 kB 393.8 kB/s eta 0:00:02
   --- ----------------------------------- 61.4/798.0 kB 252.2 kB/s eta 0:00:03
   ---- ---------------------------------- 81.9/798.0 kB 270.5 kB/s eta 0:00:03
   ---- ---------------------------------- 92.2/798.0 kB 262.6 kB/s eta 0:00:03
   ----- -------------------------------- 122.9/798.0 kB 300.4 kB/s eta 0:00:03
   ----- -------------------------------- 122.9/798.0 kB 300.4 kB/s eta 0:00:03
   ------ ------------------------------- 143.4/798.0 kB 315.4 kB/s eta 0:00:03
   -------- ----------------------------- 174.1/798.0 kB 

In [2]:
pip install psycopg2


   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   - -------------------------------------- 0.0/1.2 MB 325.1 kB/s eta 0:00:04
   - -------------------------------------- 0.0/1.2 MB 325.1 kB/s eta 0:00:04
   - -------------------------------------- 0.0/1.2 MB 196.9 kB/s eta 0:00:06
   -- ------------------------------------- 0.1/1.2 MB 252.2 kB/s eta 0:00:05
   -- ------------------------------------- 0.1/1.2 MB 286.7 kB/s eta 0:00:04
   --- ------------------------------------ 0.1/1.2 MB 291.5 kB/s eta 0:00:04
   ---- ----------------------------------- 0.1/1.2 MB 327.4 kB/s eta 0:00:04
   ---- ----------------------------------- 0.1/1.2 MB 355.0 kB/s eta 0:00:03
   ----- ---------------------------------- 0.2/1.2 MB 350.7 kB/s eta 0:00:03
   ------- -----------

In [5]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.llms.openai import OpenAI
from langchain.agents import AgentExecutor
import os
from langchain_community.llms import AzureOpenAI
os.environ["OPENAI_API_TYPE"] = "azure"
import openai



In [ ]:
openai.api_type = "azure"
openai.api_base = "https://like-card-test.openai.azure.com/"
openai.api_version = "2023-07-01-preview"
openai.api_key = "85889c7998dd4adb9a4c89abe56b1242"
openai.azure_endpoint="https://like-card-test.openai.azure.com/openai/deployments/gpt-35-turbo/chat/completions?api-version=2023-07-01-preview"

In [23]:
import os

os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2023-07-01-preview"
os.environ["OPENAI_API_BASE"] = "https://like-card-test.openai.azure.com/"
os.environ["OPENAI_API_KEY"] = "85889c7998dd4adb9a4c89abe56b1242"
os.environ["OPENAI_API_ENDPOINT"]="https://like-card-test.openai.azure.com/openai/deployments/gpt-35-turbo/chat/completions?api-version=2023-07-01-preview"

In [ ]:
f = open("Credintials.txt", "r")
email = None
password = None
for i, line in enumerate(f):
    if i == 0:
        email = line.strip()  # Assign the first line to email variable
    elif i == 1:
        password = line.strip()  # Assign the second line to password variable

f.close()

In [30]:
server = 'anmx3c2yurjethctgyba6xtuaq-vkvbn7t4322edeguc3ufhjtjeu.datawarehouse.pbidedicated.windows.net'
database = 'likeCard'
driver='ODBC Driver 18 for SQL Server'
user= email # use %40 inplace of @
pass1=password # use %40 inplace of @   
db_string =f'mssql+pyodbc://{user}:{pass1}@{server}/{database}?driver={driver}&Trusted_Connection=no&Authentication=ActiveDirectoryInteractive'
dp=SQLDatabase.from_uri(db_string)


In [62]:
llm = AzureOpenAI( model_name="gpt-35-turbo-instruct",deployment_name="gpt-35-turbo-instruct",temperature=0)

C:\Users\moham\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The class `AzureOpenAI` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.AzureOpenAI instead.
  warn_deprecated(


In [63]:
toolkit = SQLDatabaseToolkit(db=dp,llm=llm)


In [64]:
agent_executor = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True 
)

C:\Users\moham\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The class `ZeroShotAgent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use create_react_agent instead.
  warn_deprecated(
C:\Users\moham\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The class `Agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [ ]:
PROMPT = """ 
Given an input question, first create sql server query to run,  
then look at the results of the query and return the answer.  
The question: {question}
"""
question = "what is the total sales for b2b"
agent_executor.run(PROMPT.format(question=question))
  

In [66]:
agent_executor.run("what is the total sales yesterday")




> Entering new AgentExecutor chain...


C:\Users\moham\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Action: sql_db_list_tables
Action Input: 
Observation: BL_Mapping_Table, Customers_DAX_table, DAX_table, GMV_Calender_DAX_table, GMV_DimDate_DAX_table, Same_period, dimDate, dimDateEX, dim_B2B_Sales_Channel, dim_B2B_Sales_Channel_new, dim_Target, dim_allusers, dim_balance, dim_categories, dim_industry, dim_log_status, dim_parent_categories, dim_payment_methods, dim_pos_sales_channel, dim_products, dim_segments, dim_stores, dim_users_b2b, dim_users_b2c, dim_users_pos, exploration, fact_b2b_trans, fact_b2c_trans, fact_points, fact_pos_trans, fact_salla_trans, fact_transactions, fact_vouchers
Thought:

C:\Users\moham\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


 I should query the schema of the fact_b2b_trans table to see what columns I can use to calculate sales.
Action: sql_db_schema
Action Input: fact_b2b_trans
Observation: 
CREATE TABLE fact_b2b_trans (
	[OrderDateTime] DATETIME2 NULL, 
	user_id INTEGER NULL, 
	order_id INTEGER NULL, 
	store_id INTEGER NULL, 
	payment_method_id INTEGER NULL, 
	bank_id INTEGER NULL, 
	product_id INTEGER NULL, 
	qty INTEGER NULL, 
	cost FLOAT(53) NULL, 
	coupon_code VARCHAR(100) COLLATE Latin1_General_100_BIN2_UTF8 NULL, 
	fees FLOAT(53) NULL, 
	amount FLOAT(53) NULL, 
	total_amount FLOAT(53) NULL, 
	id INTEGER NOT NULL, 
	coupon_discount FLOAT(53) NULL, 
	time INTEGER NULL, 
	[OrderTimeKey] VARCHAR(50) COLLATE Latin1_General_100_BIN2_UTF8 NULL, 
	[Hour] INTEGER NULL, 
	[Business_line] VARCHAR(10) COLLATE Latin1_General_100_BIN2_UTF8 NULL, 
	CONSTRAINT [PK_id_trans_b2b] PRIMARY KEY (id)
)

/*
3 rows from fact_b2b_trans table:
OrderDateTime	user_id	order_id	store_id	payment_method_id	bank_id	product_id	qty	c

C:\Users\moham\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


 I should query the fact_b2b_trans table to get the total sales yesterday.
Action: sql_db_query
Action Input: SELECT SUM(total_amount) FROM fact_b2b_trans WHERE OrderDateTime >= '2023-12-25 00:00:00' AND OrderDateTime < '2023-12-26 00:00:00'
Observation: [(1164526.1309999377,)]
Thought:

C:\Users\moham\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langchain_core\_api\deprecation.py:189: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


 The total sales yesterday was 1,164,526.13
Final Answer: 1,164,526.13<|im_end|>

> Finished chain.


{'input': 'what is the total sales yesterday',
 'output': '1,164,526.13<|im_end|>'}